In [74]:
import pandas as pd
import numpy as np

### Wave 2 Data Extraction

In [75]:
today = input("Today's date: ")

Today's date: 2020-12-18


In [76]:
# Read the raw combined covid data file
covid_combined = pd.read_excel("combined-covid.xlsx")

# Drop the city and bed column
covid_combined = covid_combined.drop(columns=["City"])
covid_combined = covid_combined.drop(columns=["Beds"])

# Replace some weird and inconsistent names
covid_combined = covid_combined.replace("Macassa  Lodge", "Macassa Lodge")
covid_combined = covid_combined.replace("Bella Senior Care Residences", "Bella Senior Care Residence")
covid_combined = covid_combined.replace("Caressant Care - Arthur", "Caressant Care Arthur Nursing Home")
covid_combined = covid_combined.replace("King City Lodge Nurisng Home", "King City Lodge Nursing Home")
covid_combined = covid_combined.replace("Roseview Manor Long Term Care Centre", "Southbridge Roseview")
covid_combined = covid_combined.replace("Ãƒâ€°lisabeth-BruyÃƒÂ¨re Residence", "Élisabeth-Bruyère Residence")

# covid_combined.head()

In [77]:
# Deal with bad data (need to be tested)
covid_combined = covid_combined.replace('<', '<5')
covid_combined = covid_combined.replace(' ', np.nan)

# Replace "<5" as 1 and fill na as 0
covid_combined = covid_combined.replace('<5', 1)
covid_combined = covid_combined.fillna(0)

In [78]:
covid_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79405 entries, 0 to 79404
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Report_Data_Extracted      79405 non-null  datetime64[ns]
 1   LTC_Home                   79405 non-null  object        
 2   Total_LTC_Resident_Cases   79405 non-null  float64       
 3   Total_LTC_Resident_Deaths  79405 non-null  int64         
 4   Total_LTC_HCW_Cases        79405 non-null  float64       
 5   Status                     79405 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 3.6+ MB


In [79]:
# Extract the wave2 homes list (Homes who got an outbreak after September 1st)
wave2_home_list = list(covid_combined.loc[covid_combined["Report_Data_Extracted"] > '2020-08-31'].query("Status == 'Active'")["LTC_Home"].unique())
len(wave2_home_list)

298

In [80]:
# Extract the wave2 historical maximum number of cases and deaths for each home
cum_max = covid_combined.loc[covid_combined["Report_Data_Extracted"] > '2020-08-31'].drop(columns=["Report_Data_Extracted", "Status"])
cum_max = cum_max.groupby(['LTC_Home']).max().reset_index()
cum_max

,LTC_Home,Total_LTC_Resident_Cases,Total_LTC_Resident_Deaths,Total_LTC_HCW_Cases
0,Afton Park Place Long Term Care Community,0.0,0,0.0
1,"Albright Gardens Homes, Incorporated",0.0,0,1.0
2,Alexander Place,0.0,0,1.0
3,Algonquin Nursing Home,1.0,0,1.0
4,Allendale,33.0,18,14.0
...,...,...,...,...
440,peopleCare A.R. Goudie Kitchener,1.0,9,0.0
441,peopleCare Hilltop Manor Cambridge,0.0,0,0.0
442,peopleCare Oakcrossing London,0.0,0,1.0
443,peopleCare Tavistock,0.0,0,1.0


In [81]:
# Extract the current number of cases and status for each home
date = pd.Timestamp(today)
cur_num = covid_combined.loc[covid_combined["Report_Data_Extracted"] == date]
cur_num = cur_num.drop(columns=["Report_Data_Extracted", "Total_LTC_Resident_Deaths"])
cur_num.columns =  ["LTC_Home", "Current_resident_cases", "Current_staff_cases", "Current_outbreak_status"]

In [82]:
cur_num.head()

,LTC_Home,Current_resident_cases,Current_staff_cases,Current_outbreak_status
78972,Afton Park Place Long Term Care Community,0.0,0.0,Resolved
78973,"Albright Gardens Homes, Incorporated",0.0,0.0,Resolved
78974,Alexander Place,0.0,0.0,Resolved
78975,Almonte Country Haven,0.0,0.0,Resolved
78976,Altamont Care Community,0.0,0.0,Resolved


In [83]:
# Extract the number of deaths for wave1
wave1_data = covid_combined.loc[covid_combined["Report_Data_Extracted"] <= '2020-08-31'][["LTC_Home", "Total_LTC_Resident_Deaths"]]
wave1_death = wave1_data.groupby("LTC_Home").max().reset_index()
wave1_death.columns = ["LTC_Home", "Total_LTC_Resident_Deaths_wave1"]
wave1_death

,LTC_Home,Total_LTC_Resident_Deaths_wave1
0,Afton Park Place Long Term Care Community,0
1,"Albright Gardens Homes, Incorporated",0
2,Alexander Place,0
3,Allendale,0
4,Almonte Country Haven,30
...,...,...
337,Yee Hong Centre - Scarborough McNicoll,0
338,peopleCare A.R. Goudie Kitchener,9
339,peopleCare Hilltop Manor Cambridge,0
340,peopleCare Oakcrossing London,0


In [84]:
# Combine cum_max, wave1_death and cur_num as one data frame
combined = cum_max.merge(wave1_death, how = "left", on = ["LTC_Home"])
combined = combined.merge(cur_num, how = "left", on = ["LTC_Home"])

# If the outbreak status is Na, then fill the Na with "Resolved" (Assumption)
combined["Current_outbreak_status"] = combined["Current_outbreak_status"].fillna("Resolved")

# If the wave1 death is Na, then fill the Na with 0.
combined["Total_LTC_Resident_Deaths_wave1"] = combined["Total_LTC_Resident_Deaths_wave1"].fillna(0)

# Create a new column of wave2 resident death
combined["Total_LTC_Resident_Deaths_wave2"] = combined["Total_LTC_Resident_Deaths"] - combined["Total_LTC_Resident_Deaths_wave1"]

# Replace 1 with "less than 5"
combined = combined.replace(1, "less than 5")

# Filter out the homes that did not have any outbreak after Sepetember 1st (Filter out the homes in wave2)
for home in combined ["LTC_Home"]:
    if home not in wave2_home_list:
        combined = combined.query("LTC_Home != @home")
        
combined = combined[['LTC_Home',
 'Total_LTC_Resident_Cases',
 'Total_LTC_HCW_Cases',
 'Total_LTC_Resident_Deaths',
 'Total_LTC_Resident_Deaths_wave1',
 'Total_LTC_Resident_Deaths_wave2',
 'Current_resident_cases',
 'Current_staff_cases',
 'Current_outbreak_status']]

In [85]:
# Deal with -1 death
combined['Total_LTC_Resident_Deaths_wave2'] = combined['Total_LTC_Resident_Deaths_wave2'].replace(-1, 0)

In [86]:
combined

,LTC_Home,Total_LTC_Resident_Cases,Total_LTC_HCW_Cases,Total_LTC_Resident_Deaths,Total_LTC_Resident_Deaths_wave1,Total_LTC_Resident_Deaths_wave2,Current_resident_cases,Current_staff_cases,Current_outbreak_status
1,"Albright Gardens Homes, Incorporated",0,less than 5,0,0,0,0,0,Resolved
2,Alexander Place,0,less than 5,0,0,0,0,0,Resolved
3,Algonquin Nursing Home,less than 5,less than 5,0,0,0,NaN,NaN,Resolved
4,Allendale,33,14,18,0,18,11,12,Active
9,Avalon Retirement Centre,0,less than 5,0,0,0,0,0,Resolved
...,...,...,...,...,...,...,...,...,...
439,York Region Newmarket Health Centre,22,15,less than 5,0,less than 5,16,8,Active
440,peopleCare A.R. Goudie Kitchener,less than 5,0,9,9,0,0,0,Resolved
442,peopleCare Oakcrossing London,0,less than 5,0,0,0,0,0,Resolved
443,peopleCare Tavistock,0,less than 5,0,0,0,0,less than 5,Active


In [87]:
combined.to_excel("Wave2-Combined.xlsx")

### Wave 1 Data Extraction

In [26]:
# Read the raw combined covid data file for wave 1
covid_combined = pd.read_excel("combined-covid.xlsx")

# Extract data for wave1
covid_combined = covid_combined.loc[covid_combined["Report_Data_Extracted"] <= '2020-08-31']

# Drop the city column
covid_combined = covid_combined.drop(columns=["City", "Beds"])

# Replace some weird names
covid_combined = covid_combined.replace("Macassa  Lodge", "Macassa Lodge")
covid_combined.head()

,Report_Data_Extracted,LTC_Home,Total_LTC_Resident_Cases,Total_LTC_Resident_Deaths,Total_LTC_HCW_Cases,Status
0,2020-04-24,Allendale,0,0,0,Resolved
1,2020-04-24,Chartwell Westmount Long Term Care Residence,0,0,0,Resolved
2,2020-04-24,Chelsey Park,0,0,0,Resolved
3,2020-04-24,Chester Village,0,0,0,Resolved
4,2020-04-24,Clarion Nursing Home,0,0,0,Resolved


In [27]:
# Replace "<5" as 1 and fill na as 0
covid_combined = covid_combined.replace('<5',1)
covid_combined = covid_combined.fillna(0)

In [28]:
# Extract the wave1 historical maximum number of cases and deaths for each home
cum_max = covid_combined.drop(columns=["Report_Data_Extracted", "Status"])
cum_max = cum_max.groupby(['LTC_Home']).max().reset_index()
cum_max

,LTC_Home,Total_LTC_Resident_Cases,Total_LTC_Resident_Deaths,Total_LTC_HCW_Cases
0,Afton Park Place Long Term Care Community,0,0,1.0
1,"Albright Gardens Homes, Incorporated",1,0,1.0
2,Alexander Place,1,0,0.0
3,Allendale,0,0,0.0
4,Almonte Country Haven,45,30,23.0
...,...,...,...,...
337,Yee Hong Centre - Scarborough McNicoll,1,0,1.0
338,peopleCare A.R. Goudie Kitchener,16,9,18.0
339,peopleCare Hilltop Manor Cambridge,0,0,1.0
340,peopleCare Oakcrossing London,1,0,1.0


In [30]:
# Extract the current number of cases and status for each home on Aug 31!!
cur_num = covid_combined.loc[covid_combined["Report_Data_Extracted"] == '2020-08-31']
cur_num = cur_num.drop(columns=["Report_Data_Extracted", "Total_LTC_Resident_Deaths"])
cur_num.columns =  ["LTC_Home", "Current_resident_cases", "Current_staff_cases", "Current_outbreak_status"]
cur_num

,LTC_Home,Current_resident_cases,Current_staff_cases,Current_outbreak_status
29054,Afton Park Place Long Term Care Community,0,0.0,Resolved
29055,"Albright Gardens Homes, Incorporated",0,0.0,Resolved
29056,Alexander Place,0,0.0,Resolved
29057,Allendale,0,0.0,Resolved
29058,Almonte Country Haven,0,0.0,Resolved
...,...,...,...,...
60758,Golden Years Nursing Home,0,0.0,Active
60759,Madonna Care Community,0,1.0,Active
60760,Mill Creek Care Centre,0,1.0,Active
60761,Residence Saint-Louis,0,1.0,Active


In [31]:
combined = cum_max.merge(cur_num, how = "left", on = ["LTC_Home"])

# If the outbreak status is Na, then fill the Na with "Resolved" (Assumption)
combined["Current_outbreak_status"] = combined["Current_outbreak_status"].fillna("Resolved")

# Replace 1 with "less than 5"
combined = combined.replace(1, "less than 5")

In [32]:
combined

,LTC_Home,Total_LTC_Resident_Cases,Total_LTC_Resident_Deaths,Total_LTC_HCW_Cases,Current_resident_cases,Current_staff_cases,Current_outbreak_status
0,Afton Park Place Long Term Care Community,0,0,less than 5,0,0,Resolved
1,"Albright Gardens Homes, Incorporated",less than 5,0,less than 5,0,0,Resolved
2,Alexander Place,less than 5,0,0,0,0,Resolved
3,Allendale,0,0,0,0,0,Resolved
4,Almonte Country Haven,45,30,23,0,0,Resolved
...,...,...,...,...,...,...,...
337,Yee Hong Centre - Scarborough McNicoll,less than 5,0,less than 5,0,0,Resolved
338,peopleCare A.R. Goudie Kitchener,16,9,18,0,0,Resolved
339,peopleCare Hilltop Manor Cambridge,0,0,less than 5,0,0,Resolved
340,peopleCare Oakcrossing London,less than 5,0,less than 5,0,0,Resolved


In [33]:
combined.to_excel("Wave1-Combined.xlsx")